In [1]:
import numpy as np
import tensorflow as tf
import etp as etp
import calculate_virtual_coordinates as cvc

C = 100 # maximum number of anchors
H = 500 # size of hidden filters
F = 2 # Final dimension of coordinates
random_seed = 23
radius = 1 # extent of possible communication
MAX_NODES = 10
learning_rate = 0.7
epochs = 10

In [2]:
# def distance(x, y, dist_matrix):
#     return np.sqrt((dist_matrix[x, 0] - dist_matrix[y, 0])**2 + (dist_matrix[x, 1] - dist_matrix[y, 1])**2) 
    
# def is_neighbour(x, y, dist_matrix):
#     return distance(x, y, dist_matrix) <= 1

def get_input(MAX_NODES): 
    cvc.generate_physical_coordinates(MAX_NODES)


def get_A_caret(adj):
    return tf.Variable(initial_value=tf.ones([MAX_NODES, MAX_NODES]), name='A_caret')

In [3]:
g = tf.Graph()

with g.as_default():
    tf.set_random_seed(random_seed)
    W_0 = tf.Variable(tf.truncated_normal(shape=(C, H),
                                      mean=0.0,
                                      stddev=0.1,
                                      dtype=tf.float32,
                                      seed=random_seed))
    
    W_1 = tf.Variable(tf.truncated_normal(shape=(H, F),
                                  mean=0.0,
                                  stddev=0.1,
                                  dtype=tf.float32,
                                  seed=random_seed))
    

    input_layer = tf.placeholder(tf.float32, [None, C], # INPUT: num_of_nodes X VC_values
                                 name='input')
    
    physical_coordinates = tf.placeholder(tf.float32, [None, F], # INPUT: num_of_nodes X VC_values
                                 name='phy_coord')
    
    adj = tf.placeholder(tf.float32, [None, None], # INPUT: num_of_nodes X VC_values
                                 name='adj')
    
    A_caret = tf.placeholder(tf.float32, [None, None], # INPUT: num_of_nodes X VC_values
                                 name='A_caret')
    
#     adj = cvc.get_adj(physical_coordinates)
#     A_caret = cvc.normalize_adj(adj + np.eye(adj.shape[0]))
    
    
    
    out_1 = tf.matmul(tf.matmul(A_caret, input_layer), W_0)
    out_2 = tf.matmul(tf.matmul(A_caret, out_1), W_1)
    
    print('out_2 shape: {}'.format(out_2.shape))
    
#     cost = etp.get_best_etp(out_2, physical_coordinates)
#     cost = etp.get_best_etp(out_2, physical_coordinates)
    
    cost = tf.losses.mean_squared_error(out_2, physical_coordinates)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    

 

Instructions for updating:
Colocations handled automatically by placer.
out_2 shape: (?, 2)
Instructions for updating:
Use tf.cast instead.


In [4]:
placeholders = {'input_layer' : input_layer, 'physical_coordinates' : physical_coordinates, 'adj' : adj, 'A_caret' : A_caret}

In [5]:
def construct_feed_dict(input_layer, physical_coordinates, adj, A_caret, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['input_layer']: input_layer})
    feed_dict.update({placeholders['physical_coordinates']: physical_coordinates})
    feed_dict.update({placeholders['adj']: adj})
#     feed_dict.update({placeholders['support'][i]: support[i] for i in range(len(support))})
    feed_dict.update({placeholders['A_caret']: A_caret})
    return feed_dict


In [6]:
with tf.Session() as sess:
    # `sess.graph` provides access to the graph used in a `tf.Session`.
    writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)

    # Perform your computation...
    for i in range(2):
        physical_coordinates, inps = cvc.get_VC(MAX_NODES)
        adj = cvc.get_adj(physical_coordinates)
        A_caret = cvc.normalize_adj(adj + np.eye(adj.shape[0]))
        feed_dict_val = construct_feed_dict(inps, physical_coordinates, adj, A_caret, placeholders)
        cost = sess.run(cost, feed_dict=feed_dict_val)

    writer.close()


DEBUG: Anchor list: [4 8 5 6 1]


TypeError: Value passed to parameter 'x' has DataType int64 not in list of allowed values: bfloat16, float16, float32, float64, complex64, complex128